<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : Générateur markovien de texte</h1>

L'objectif est d'écrire un programme capable de générer automatiquement un texte le plus cohérent possible. La langue utilisée pour l'exercice est l'anglais.

Le programme utilise les dictionnaires pour modeler, puis générer, un texte. 

## Génération markovienne de texte
Voici l'idée principale : la langue anglaise possède de nombreuses structures. Les mots ont tendance à apparaître seulement dans certaines séquences. Les règles grammaticales spécifient les combinaisons autorisées dans différentes partie d'un texte. La phrase : "The cat climbs the stairs" constitue une suite de mots grammaticalement correcte. Au contraire de la phrase : "Stairs the the climbs cat". De plus la sémantique (la signification d'un mot ou d'une phrase), limite davantage la combinaison possible des mots. "The stairs clim the cat" est une phrase grammaticalement correcte mais qui n'a aucun sens et il est très peu probable de rencontrer cet enchaînement de mot en pratique.

Même sans connaître les règles formelles de l'anglais ou le sens des mots anglais, on peut avoir une idée de ce qui constitue une suite cohérente de mots simplement en observant un texte anglais correct et en relevant les combinaisons de mots du texte. Puis, en se basant sur ces observations, on peut générer de nouvelles phrases en selectionnant des mots aléatoirement en accord avec les enchaînements de mots communement observés.  

Par exemple, considérons le texte : 

"I love roses and carnations. I hope I get roses for my birthday."

* Si on commence par selectionner le mot "I", on remarque que "I" peut être suivi par "love", "hope" et "get" avec les mêmes probabilités dans ce texte. 
* On selectionne l'un de ces mots, aléatoirement, pour compléter notre phrase. On obtient, par exemple, "I get".
* On peut répéter le procédé avec le mot "get", en sélectionnant (obligatoirement) le mot "roses" pour compléter la phrase.
* En continuant le procédé, on obtient la phrase "I get roses and carnations". On remarque que la phrase est grammaticalement correcte. Ce ne sera pas toujours le cas.

Nous aurions pu obtenir différentes phrases. Par exemple, "I love roses for my birthday," et "I get roses for my birthday".

De façon plus formelle, le procédé utilisé pour générer ces phrase est appelé **processus de Markov du premier ordre**. Un processus de Markov du premier ordre est un procédé dans lequel l'état au temps $t+1$ (c'est-à-dire le prochain mot) ne dépend que de l'état au temps $t$ (c'est-à-dire la mot précédent).  
Dans un processus de Markov du second ordre, le prochain mot dépend des deux mots précédent, etc...

Notre exemple précédent était un processus de Markov du premier ordre car le choix du mot suivant ne dépend que du mot courant.  
On remarque que la valeur du mot suivant est indépendant de la position du mot (dans la phrase). Ainsi, cela ne change rien si nous choisissons le 2eme ou le 92eme mot (le choix ne dépendra que du 1er, ou 91eme mot).

## Analiseur et générateur de texte
Dnas un premier temps, il faut implémenter un générateur de texte markovien du premier ordre.  
Il faut donc deux fonctions :
1. Une pour traiter un fichier (texte) et créer un dictionnaire de transition
2. Une autre pour générer un nouveau texte (à partir du dictionnaire créé précédemment.

On suppose ici que le code considérera que les mots sont différents s'ils n'ont pas la même casse ou suivi d'une ponctuation. AInsi les mots `spam`, `Spam` et `spam.` sont trois mots différents.


## La fonction `createDictionary`
La fonction `createDictionary(filename)` prend, en paramètre, une chaîne de caractère correspondant au nom du fichier texte qui contient le texte à analyser. Elle renvoie un dictionnaire dont les clés sont les mots encontrés dans le texte et la valeurs associée à une clé est une liste de mots qui peuvent, grammaticalement, suivre le mot correspondant à la clé.  
Il faut également déterminer un moyen de conserver les information sur la fréquence des mots. Ainsi si le mot "cheese" est suivi du mot "pizza", deux fois plus que le mot "sandwich", le dictionnaire doit refléter cette tendance. Par exemple, on pourra garder un mot plusieurs fois dans la liste.

Le dictionnaire renvoyé par `createDictionary` peremettra de choisir le mot au temps $t+1$ en fonction du mot au temps $t$.  

Mais comment choisir le premier mot, puisqu'on n'a pas de mot précédent dans ce cas? Pour gérer cette situation, le dictionnaire doit inclure le caractère `'$'` qui représente le symbole de départ. Le premier mot dans le fichier doit suivre ce caractère.  


De la même manière, chaque mot du fichier qui suit un symbole de fin de phrase doit suivre ce caractère.  
Un sumbole de fin de phrase sera définie par un saut de ligne, un espace après un point `.`, une interrogation `?` ou une exclamation `!`. La manière la plus simple est de vérifier `w[-1]`. One s'intressera qu'à `.`, `?` ou `!`.

Pour tester votre code, considérons le fichier [`t.txt`](Fichiers/t.txt), contenant le texte :

    A B A. A B C. B A C. C C C.

Voici, ce que vous devriez obtenir :

```
>>> d1 = createDictionary('t.txt')
>>> d1
{'A': ['B', 'B', 'C.'], 'C': ['C', 'C.'], 
 'B': ['A.', 'C.', 'A'], '$': ['A', 'A', 'B', 'C']}
```


Les éléments de chaque liste ne sont pas nécessairement dans le même ordre mais ils doivent apparaître en même quantité pour chacune des quatre clés : `'A'`, `'C'`, `'B'`, and `'$'`.

Considérons le fichier [`a.txt`](Fichiers/a.txt), contenant le texte.

    I like poptarts and 42 and spam.
    Will I get spam and poptarts for
    the holidays? I like spam poptarts!

Voici, ce que vous devriez obtenir :

```
>>> d2 = createDictionary('a.txt')
>>> d2
{'and': ['42', 'spam.', 'poptarts'], 
'$': ['I', 'Will', 'I'], 
'for': ['the'], 
'get': ['spam'], 
'I': ['like', 'get', 'like'], 
'spam': ['and', 'poptarts!'], 
'42': ['and'], 
'Will': ['I'], 
'poptarts': ['and', 'for'], 
'the': ['holidays?'], 
'like': ['poptarts', 'spam']}
```


## La fonction `generateText`
La fonction `generateText(d, n)` prend, en paramètres, un dictionnaire de transitions `d` (générer par la fonction `createDictionary`) et un entier naturel `n`. Elle renvoie un texte de `n` mots.

Le premier mot doit être choisi, aléatoirement, parmi ceux qui suivent le caractère de départ `'$'`.

On pourra utiliser la fonction `random.choice`.

Le second mot sera choisi, aléatoirement, dans la liste de mots qui peuvent suivre le premier, etc...

Lorsqu'un mot choisi termine par un point `.`, un point d'interrogation `?` ou un point d'exclamation `!`, la fonction `generateText` doit le détecter et démarrer une nouvelle phrase, en choisissant, de nouveua, un mot aléatoirement parmi ceux qui suivent `'$'`.  
Il ne faut pas inclure le caractère `'$'` dans le texte, il servira de marqueur interne pour la fonction.

Pour ce problème, il na faut pas enlever la ponctuation dans les mots du texte, lors de l'analyse. Il faut laisser la ponctuation, telle qu'elle apparaît dans le texte (il ne faut pas se préoccuper du fait que le texte ne sera pas cohérent pour la ponctuation.  
Le texte généré ne sera pas parfait, mais il pourra s'avérer de bonne qualité.

On peut essayer d'éviter d'avoir une ponctuation isolée pour le mot de départ. Pour cela, le mieux est de gérer le cas en vérifant si le premier mot fini par une ponctuation et si c'est le cas, commencer une nouvelle phrase. En particulier, si l'on rencontre un mot qui n'apparaît qu'en fin de phrase (qui n'est donc suivi d'aucun mot), on recommence à créer une phrase avec le symbole `'$'`.

Voici deux exemples qui utilisent le dictionnaire `d1`, construit précédemment. Les résultats pourront être différents du fait du processus aléatoire.

```
>>> generateText(d1, 20)
B C. C C C. C C C C C C C C C C C. C C C. A

>>> generateText(d1, 20)
A B A. C C C. B A B C. A C. B A. C C C C C C.
```


## Génération d'un texte
Pour terminer cette activité, il faut trouver un fichier texte interessant, créer un modèle de Markov du premier ordre et générer un texte.

Il faut, au minimum 500 mots, dans le texte.

Vous pouvez choisir le texte que vous souhaitez pour construire votre modèle. Vous pouvez choisir des [textes de Shakespeare](http://shakespeare.mit.edu/), des [livres](https://www.gutenberg.org/), des paroles de chanson, des discours, vos propres textes, ...